In [14]:
# import sys
# !{sys.executable} -m pip install python-dotenv
# !{sys.executable} -m pip install google-api-python-client

In [15]:
# !pip install iteration_utilities

In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import re
from tqdm import tqdm
from iteration_utilities import unique_everseen
import csv

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [3]:
# create a service object for interacting with the YouTube Data API
youtube = build("youtube", "v3", developerKey=api_key, cache_discovery=False)

In [4]:
# TEST
# link = "https://www.youtube.com/watch?v=36N1Bz7qW0A"
# request = youtube.commentThreads().list(part='id,replies,snippet', videoId=extract_video_id(link))
# response = request.execute()
# print(response)

In [13]:
def extract_video_id(url):
    video_id = None
    if "youtu.be" in url:
        video_id = url.split("/")[-1]
    else:
        query_string = url.split("?")[-1]
        parameters = query_string.split("&")
        for param in parameters:
            if param.startswith("v="):
                video_id = param[2:]
                break

    return video_id

In [6]:
comments = []

In [7]:
def make_csv(comments, channelID=None):
    header = comments[0].keys()

    if channelID:
#         filename = f'comments_{channelID}_{today}.csv'
        filename = f'comments.csv'
    else:
#         filename = f'comments_{today}.csv'
        filename = f'comments.csv'

    with open(filename, 'a', encoding='utf8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=header, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(comments)

In [8]:
def process_comments(response_items, csv_output=False):

    for res in response_items:

        # loop through the replies
        if 'replies' in res.keys():
            for reply in res['replies']['comments']:
                comment = reply['snippet']
                comment['commentId'] = reply['id']
                comments.append(comment)
        else:
            comment = {}
            comment['snippet'] = res['snippet']['topLevelComment']['snippet']
            comment['snippet']['parentId'] = None
            comment['snippet']['commentId'] = res['snippet']['topLevelComment']['id']

            comments.append(comment['snippet'])

    if csv_output:
         make_csv(comments)
    
    print(f'Finished processing {len(comments)} comments.')
    return comments

In [9]:
def comment_threads(videoID, to_csv=False):
    
    comments_list = []
    
    request = youtube.commentThreads().list(
        part='id,replies,snippet',
        videoId=videoID,
    )
    response = request.execute()
    comments_list.extend(process_comments(response['items']))

    # if there is nextPageToken, then keep calling the API
    while response.get('nextPageToken', None):
        request = youtube.commentThreads().list(
            part='id,replies,snippet',
            videoId=videoID,
            pageToken=response['nextPageToken']
        )
        response = request.execute()
        comments_list.extend(process_comments(response['items']))

    comments_list = list(unique_everseen(comments_list))

    print(f"Finished fetching comments for {videoID}. {len(comments_list)} comments found.")
    
    if to_csv:
#         make_csv(comments_list, videoID)
        make_csv(comments_list)
    return comments_list

In [17]:
video_ids = [
    "9g3692xrYog",
    "-L-xfr3pyMM",
    "tC0TWLQwxlg",
    "XAD-xUbncew",
    "xDCtlAGjA0c",
    "YFpYNRX67hk",
    "Cp48NX8clAA",
    "ThvNwfGwaOE",
    "S3WE134o-mo",
    "ikTln_LOADg",
    "d5FKIh21kVQ",
    "fOoMDPcy-ao",
    "AfS_X2bfAJg",
    "YVxSNRhVypA",
    "ttDpr-6_GBI",
    "ur4uq7_ewUQ",
    "5gay09DQdQI",
    "f-RaA60FUrw",
    "TwqmICVZ7xQ",
    "WrsZJxpZcH0",
    "2yTT_gsiXL0",
    "_ShclHTKJoY"
]

In [18]:
%%time
if __name__ == '__main__':
    # get comments
#     response = comment_threads(videoID='7Kt6ouYqacQ', to_csv=True)
#     print(response)
    for video_id in tqdm(video_ids, desc='Processing videos'):
        response = comment_threads(videoID=video_id, to_csv=True)

Processing videos:   0%|                                                                        | 0/22 [00:00<?, ?it/s]

Finished processing 20 comments.
Finished processing 40 comments.
Finished processing 60 comments.
Finished processing 81 comments.


Processing videos:   5%|██▉                                                             | 1/22 [00:01<00:37,  1.77s/it]

Finished processing 97 comments.
Finished fetching comments for 9g3692xrYog. 97 comments found.
Finished processing 117 comments.
Finished processing 140 comments.


Processing videos:   9%|█████▊                                                          | 2/22 [00:02<00:23,  1.17s/it]

...

Finished processing 1417 comments.
Finished fetching comments for WrsZJxpZcH0. 1417 comments found.
Finished processing 1437 comments.
Finished processing 1457 comments.
Finished processing 1477 comments.
Finished processing 1500 comments.
Finished processing 1529 comments.
Finished processing 1549 comments.
Finished processing 1581 comments.
Finished processing 1620 comments.
Finished processing 1629 comments.


Processing videos:  95%|████████████████████████████████████████████████████████████▏  | 21/22 [00:26<00:02,  2.43s/it]

Finished fetching comments for 2yTT_gsiXL0. 1629 comments found.
Finished processing 1649 comments.
Finished processing 1669 comments.
Finished processing 1689 comments.
Finished processing 1709 comments.
Finished processing 1729 comments.
Finished processing 1749 comments.
Finished processing 1769 comments.
Finished processing 1790 comments.
Finished processing 1810 comments.
Finished processing 1836 comments.


Processing videos: 100%|███████████████████████████████████████████████████████████████| 22/22 [00:31<00:00,  1.45s/it]

Finished processing 1852 comments.
Finished fetching comments for _ShclHTKJoY. 1852 comments found.
CPU times: total: 8.33 s
Wall time: 32 s


In [14]:
def extract_shorts_id(url):
    if 'youtube.com/shorts/' in url:
        return url.split('youtube.com/shorts/')[1].strip('/')
    else:
        return None

In [11]:
# smriti irani
links = [
    'https://www.youtube.com/watch?v=9g3692xrYog',
    'https://www.youtube.com/watch?v=-L-xfr3pyMM', 
    'https://www.youtube.com/watch?v=tC0TWLQwxlg',
    'https://www.youtube.com/watch?v=XAD-xUbncew',
    'https://www.youtube.com/watch?v=xDCtlAGjA0c'
]
short_links = [
    "https://www.youtube.com/shorts/YFpYNRX67hk",
    "https://www.youtube.com/shorts/Cp48NX8clAA",
    "https://www.youtube.com/shorts/ThvNwfGwaOE",
    "https://www.youtube.com/shorts/S3WE134o-mo",
    "https://www.youtube.com/shorts/ikTln_LOADg",
    "https://www.youtube.com/shorts/d5FKIh21kVQ",
    "https://www.youtube.com/shorts/fOoMDPcy-ao",
    "https://www.youtube.com/shorts/AfS_X2bfAJg",
    "https://www.youtube.com/shorts/YVxSNRhVypA",
    "https://www.youtube.com/shorts/ttDpr-6_GBI",
    "https://www.youtube.com/shorts/ur4uq7_ewUQ",
    "https://www.youtube.com/shorts/5gay09DQdQI",
    "https://www.youtube.com/shorts/f-RaA60FUrw",
    "https://www.youtube.com/shorts/TwqmICVZ7xQ",
    "https://www.youtube.com/shorts/WrsZJxpZcH0",
    "https://www.youtube.com/shorts/2yTT_gsiXL0",
    "https://www.youtube.com/shorts/_ShclHTKJoY"
]

In [15]:
for l in short_links:
#     link = extract_video_id(l)
    link = extract_shorts_id(l)
    print(link)

YFpYNRX67hk
Cp48NX8clAA
ThvNwfGwaOE
S3WE134o-mo
ikTln_LOADg
d5FKIh21kVQ
fOoMDPcy-ao
AfS_X2bfAJg
YVxSNRhVypA
ttDpr-6_GBI
ur4uq7_ewUQ
5gay09DQdQI
f-RaA60FUrw
TwqmICVZ7xQ
WrsZJxpZcH0
2yTT_gsiXL0
_ShclHTKJoY
